In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string

a)remove all duplicates from sheet in excel using DATA>remove duplicates
b)using excel concatenate function joined "AgentCorrected_CatName_Primary" and "AgentCorrected_Integer_Primary" column with comma and new target column name is "primary".


In [2]:

df2_cicd=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/TaxML-CICD - Prod_Data_03_01.csv')

In [6]:
def preprocess_text(message):

    new_stopwords=['ml','oz','pk','grocery','lb']
    stpwrd = nltk.corpus.stopwords.words('english')
    stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]','', message.lower())
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message )
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    #print("message is : ",message)
    return message

In [7]:
df = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/jina_final.csv', encoding='utf8',engine='python',usecols=['Item','Description','establishment_type','primary'])
df1=df.sample(frac=1, random_state=42)
df1 = df1.fillna('')
df1['input_str'] = df1[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df1['cleanText']=df1['input_str'].map(lambda s:preprocess_text(s)) 
df1 = df1.reset_index(drop=True)
X=df1[['Item','Description','establishment_type','cleanText']]
Y_primary=df1['primary']
#Train test split with stratified sampling for evaluation
X_train, X_test, y_train_primary, y_test_primary = train_test_split(X,
                                                    Y_primary,
                                                    test_size = .20, 
                                                    random_state = 42
                                                    )

df2 = df2_cicd
df2['input_str'] = df2[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df2['cleanText']=df2['input_str'].map(lambda s:preprocess_text(s))
X_cicd=df2[['Item','Description','establishment_type','cleanText']]

Y_primary_cicd=df2['primary']
X_train_cicd, X_test_cicd, y_train_primary_cicd, y_test_primary_cicd = train_test_split(X_cicd,
                                                    Y_primary_cicd,
                                                    test_size = .20, 
                                                    random_state = 42
                                                    )
X_train_final_1 = X_train.append(X_train_cicd)
X_test_final_1 = X_test.append(X_test_cicd)
X_train_final=X_train['cleanText'].append(X_train_cicd['cleanText'])
y_train_final_primary=y_train_primary.append(y_train_primary_cicd)
X_test_final=X_test['cleanText'].append(X_test_cicd['cleanText'])
y_test_final_primary=y_test_primary.append(y_test_primary_cicd)


split and save  data for train and test

In [10]:
train_size = 0.8
train_end = int(len(df1)*train_size)
df_train = df1[:train_end]
df_test = df1[train_end:]
train_end_cicd = int(len(df2)*train_size)
df2_train = df2[:train_end_cicd]
df2_test = df2[train_end_cicd:]
df2_train = df2_train[['Item','Description','establishment_type','primary','input_str','cleanText']]
df2_test = df2_test[['Item','Description','establishment_type','primary','input_str','cleanText']]
X_train_save = df_train.append(df2_train)
X_test_save = df_test.append(df2_test)
X_train_save['label'] = 'train'
X_test_save['label'] = 'test'
X_data = X_train_save.append(X_test_save)
X_data.to_csv('df_traintestdata_03-01-22.csv')

In [8]:
X_train_final_1

,Item,Description,establishment_type,cleanText
74328,"Exotico Blanco, 750mL bottle (40% ABV)",,GROCERY,exotico blanco bottle abv
93071,Cortas Halva Original (16 oz),,GROCERY,cortas halva original
160229,"Meiomi Pinot Noir, 750mL wine (13.7% ABV)",,GROCERY,meiomi pinot noir wine abv
33442,Ito en Peach Veggie Shot,30.36 fluid ounces.,GROCERY,ito en peach veggie shot fluid ounce
129506,"Val Di Giulia Barbaresco, 750mL italian red wi...",,GROCERY,val di giulia barbaresco italian red wine abv
...,...,...,...,...
11964,"R&R Canadian Reserve Whiskey, 750mL (0.0% ABV)",NaN,GROCERY,rr canadian reserve whiskey abv
5191,"Cazadores Tequila Reposado 750mL, 1 tequila an...","91 pts beverage testing institute. Pale, silve...",LIQUOR,cazadores tequila reposado tequila mezcal abv ...
5390,American Dream Roll,"Spicy tuna, salmon, eel, shrimp, avocado, and ...",DEFAULT,american dream roll spicy tuna salmon eel shri...
860,"Vita Salmon, Nova, Atlantic, Premium, Sliced, ...",NaN,GROCERY,vita salmon nova atlantic premium sliced smoked


In [10]:
result=X_test_final_1

rf = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')),
       ('tfidf', TfidfTransformer(use_idf=True)),
       ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1)),
      ])
rf.fit(X_train_final, y_train_final_primary)
y_pred= rf.predict(X_test_final)
result['original_cat_primary']=y_test_final_primary
result['prediction_cat_primary']=y_pred
result['prediction_cat_primary_confscore']=rf.predict_proba(X_test_final).max()
output={'accuracy':accuracy_score(y_pred,y_test_final_primary),'precision_score':precision_score(y_pred,y_test_final_primary,average='macro'),'recall_score':recall_score(y_pred,y_test_final_primary,average='macro')
,'f1_score':f1_score(y_pred,y_test_final_primary,average='macro')}
result['confusion_matrix_primary']=str(output)



/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/envs/py365/lib/py

In [11]:

misclassifications_primary= result.loc[result['original_cat_primary']!=result['prediction_cat_primary']]

"{'accuracy': 0.8146149522203485, 'precision_score': 0.6027551606729535, 'recall_score': 0.7097466761674291, 'f1_score': 0.6327463859326979}"

saving model to local

In [11]:
import pickle
# save the model to disk
filename_primary= 'finalized_model_rf_primary-03-01-22_allprod.sav'
pickle.dump(rf, open(filename_primary, 'wb'))


In [11]:
print(result['confusion_matrix_primary'][0].values)

["{'accuracy': 0.8191786065842721, 'precision_score': 0.5699239468402072, 'recall_score': 0.6640538003073417, 'f1_score': 0.5976556844775389}"
 "{'accuracy': 0.8191786065842721, 'precision_score': 0.5699239468402072, 'recall_score': 0.6640538003073417, 'f1_score': 0.5976556844775389}"]


{'accuracy': 0.8198348463305261, 'precision_score': 0.5943177708372124, 'recall_score': 0.6808857187011089, 'f1_score': 0.6211879572499776}

In [ ]:
result

In [54]:
def split_cat(category):
    category_list=category.split(',')
    if len(category_list)==1:      
        primary_category=category_list[0]
        secondary_category=category_list[0]
    if len(category_list)==2:      
        primary_category=category_list[0]
        if category_list[1] in ['TEMP_HEATED','TEMP_COLD','TEMP_UNHEATED']:
              secondary_category=category_list[0]
        else:
              secondary_category=category_list[1]
    if len(category_list)>2:      
        primary_category=category_list[0]
        secondary_category=category_list[1]
    else:
        pass
        
    return primary_category,secondary_category
def split_int(integer):
    str_int=str(integer)
    integer_list=str_int.split(',')
    if len(integer_list)==1:      
        primary_integer=integer_list[0]
        secondary_integer=integer_list[0]
    if len(integer_list)==2:      
        primary_integer=integer_list[0]
        if integer_list[1]=='1':
            secondary_integer=integer_list[0]
        else:
            secondary_integer=integer_list[1]
    if len(integer_list)>2:      
        primary_integer=integer_list[0]
        secondary_integer=integer_list[1]
    else:
         pass
        
    return primary_integer,secondary_integer
def combine(category,integer):
    return category+","+integer

In [55]:
df['primary_cat']=df['Agent Corrected CAT Name'].map(lambda x:split_cat(x)[0])
df['secondary_cat']=df['Agent Corrected CAT Name'].map(lambda x:split_cat(x)[1])
df['primary_int']=df['Agent Corrected Integer'].map(lambda x:split_int(x)[0])
df['secondary_int']=df['Agent Corrected Integer'].map(lambda x:split_int(x)[1])
df['primary_int_prediction']=df['Integer'].map(lambda x:split_int(x)[0])
df['primary']=df[['primary_cat','primary_int']].apply(lambda x:combine(*x),axis=1)
df['secondary']=df[['secondary_cat','secondary_int']].apply(lambda x:combine(*x),axis=1)

In [63]:
df.to_csv('TaxML-CICD - Prod_Data_after_preprocess.csv')

In [64]:
df

,Unnamed: 0,UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,CAT Name,Integer,Confidence Score,...,Agent Corrected Integer,CAT NAME_ ValidationScore [0-100],Inetger_ValidationScore[0-100],primary_cat,secondary_cat,primary_int,secondary_int,primary_int_prediction,primary,secondary
0,NaN,2198c3e7-9b2f-441e-9e24-813d70f3bf26:9abcba67-...,2198c3e7-9b2f-441e-9e24-813d70f3bf26,9abcba67-3a79-41a5-987a-034cae6a0ff9,Pure Happiness,A sunny sunflower bouquet gets an autumnal spi...,FLOWERS,CAT_TPP,531,0.75,...,531,100,100,CAT_TPP,CAT_TPP,531,531,531,"CAT_TPP,531","CAT_TPP,531"
1,NaN,2198c3e7-9b2f-441e-9e24-813d70f3bf26:03a01438-...,2198c3e7-9b2f-441e-9e24-813d70f3bf26,03a01438-c236-40bd-a6d1-9dff278d67e3,Silver Snow Bouquet,"Like a quiet walk through a snowy forest, this...",FLOWERS,CAT_TPP,531,0.85,...,531,100,100,CAT_TPP,CAT_TPP,531,531,531,"CAT_TPP,531","CAT_TPP,531"
2,NaN,2198c3e7-9b2f-441e-9e24-813d70f3bf26:14312ee6-...,2198c3e7-9b2f-441e-9e24-813d70f3bf26,14312ee6-2324-4f0b-81b7-f1e4cb7cf454,Beautiful in Blue,Brighten the home with the beauty of bright bl...,FLOWERS,CAT_TPP,531,0.90,...,531,100,100,CAT_TPP,CAT_TPP,531,531,531,"CAT_TPP,531","CAT_TPP,531"
3,NaN,2198c3e7-9b2f-441e-9e24-813d70f3bf26:6cb0e71e-...,2198c3e7-9b2f-441e-9e24-813d70f3bf26,6cb0e71e-586d-4fd9-a71a-5dcd86e6f920,Blush Life Bouquet,Put a spring in their step with this beautiful...,FLOWERS,CAT_TPP,531,0.85,...,531,100,100,CAT_TPP,CAT_TPP,531,531,531,"CAT_TPP,531","CAT_TPP,531"
4,NaN,2198c3e7-9b2f-441e-9e24-813d70f3bf26:1dbd5b6a-...,2198c3e7-9b2f-441e-9e24-813d70f3bf26,1dbd5b6a-0e09-46e8-ad34-54b996c53d57,Red Tulips,Call ahead for this arrangement before orderin...,FLOWERS,CAT_TPP,531,0.90,...,531,100,100,CAT_TPP,CAT_TPP,531,531,531,"CAT_TPP,531","CAT_TPP,531"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,16-12-2021 1:51 AM,61ff9153-b582-5003-8680-466f512bb8e1:58739636-...,61ff9153-b582-5003-8680-466f512bb8e1,58739636-cef8-4d82-aa5b-2689dac0b482,Misko (1 lb),NaN,GROCERY,"CAT_SOFT_DRINK,CAT_PREPACKAGED_FOOD_PASTA","112,737",0.41,...,737,0,0,CAT_PREPACKAGED_FOOD_PASTA,CAT_PREPACKAGED_FOOD_PASTA,737,737,112,"CAT_PREPACKAGED_FOOD_PASTA,737","CAT_PREPACKAGED_FOOD_PASTA,737"
723,16-12-2021 1:51 AM,df29da39-f17b-5114-b9d7-1f67953a5e53:5781fe4d-...,df29da39-f17b-5114-b9d7-1f67953a5e53,5781fe4d-20f2-4c6e-985e-2bc359d57646,Chunky Chocolates (2.4 oz),Chunks of chocolate with premium ingredients m...,GROCERY,"CAT_CONFECTIONARY,CAT_CHOCOLATE","707,706",0.30,...,707,0,0,CAT_CONFECTIONARY,CAT_CONFECTIONARY,707,707,707,"CAT_CONFECTIONARY,707","CAT_CONFECTIONARY,707"
724,16-12-2021 1:51 AM,8fed2af5-d154-444a-aa96-0992a845c008:17e6f169-...,8fed2af5-d154-444a-aa96-0992a845c008,17e6f169-5a76-4b69-9519-cfac5dd4f420,"Essential Everyday Salsa, Restaurant Style, an...",NaN,GROCERY,CAT_PREPACKAGED_FOOD_CONDIMENTS,740,0.65,...,740,100,100,CAT_PREPACKAGED_FOOD_CONDIMENTS,CAT_PREPACKAGED_FOOD_CONDIMENTS,740,740,740,"CAT_PREPACKAGED_FOOD_CONDIMENTS,740","CAT_PREPACKAGED_FOOD_CONDIMENTS,740"
725,16-12-2021 1:51 AM,f0fba24f-ba3a-53eb-b35e-97efaaf4a468:63cdb229-...,f0fba24f-ba3a-53eb-b35e-97efaaf4a468,63cdb229-8333-46c1-8cfc-bcc94764d91d,"Woodbridge Chardonnay, 1.5L white wine (13.5% ...",NaN,GROCERY,CAT_WINE,534,0.68,...,534,100,100,CAT_WINE,CAT_WINE,534,534,534,"CAT_WINE,534","CAT_WINE,534"


In [57]:
df_test = df.loc[df['Inetger_ValidationScore[0-100]'] == 0]

In [58]:
len(df_test)

403

In [60]:
df2=df_test[df_test['primary_int_prediction']==df_test['primary_int']]
len(df2)#no of correct prediction

73

169000 total data b4 traing + 403 rows
exp01:169000+403 rows
exp02:169000+727 rows
